# HealthAIBot

In [1]:
# export env variables before running this block
import os

class KEYS:
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "your-openai-api-key")
    TAVILY_API_KEY = os.getenv("TAVILY_API_KEY", "your-tavily-api-key")

In [7]:
# /usr/bin/env python3
# healthAiBot/healthaibot/utils/utils.py
"""
Utility functions for HealthBot operations.
"""

from pydantic import BaseModel, Field
from typing import List, Optional, Any, Dict

from langchain_openai import ChatOpenAI
from langchain_ollama.chat_models import ChatOllama


class HealthBotState(BaseModel):
    messages: List[Dict[str, str]] = Field(default_factory=list)
    topic: Optional[str] = None
    focus: Optional[str] = None
    search_results: Optional[str] = None
    summary: Optional[str] = None
    question: Optional[str] = None
    quiz_question: Optional[str] = None
    quiz_answer: Optional[str] = None
    quiz_grade: Optional[str] = None
    grading: Optional[str] = None
    continue_flag: Optional[str] = None
    previous_questions: List[str] = Field(default_factory=list)
    tool_call_events: List[Any] = Field(default_factory=list)
    llm: Optional[Any] = None


class HealthBotUtils:
    """
    Utility functions for HealthBot operations.
    """
    def __init__(
        self,
        llm_type: str,
        model_name: str,
        temperature: float = 0.7,
    ) -> None:
        """
        Initialize the HealthBotUtils class.
        Parameters:
            llm_type: Type of LLM to use ('openai' or 'ollama').
            model_name: Name of the model to use.
            temperature: Sampling temperature for the LLM.
        """
        self.llm_type = llm_type
        self.model_name = model_name
        self.temperature = temperature

    def get_llm(
        self,
    ) -> ChatOpenAI | ChatOllama:
        """
        Get the LLM instance based on the specified type.
        Returns:
            An instance of ChatOpenAI or ChatOllama.
        """
        if self.llm_type == "openai":
            return ChatOpenAI(
                model=self.model_name,
                temperature=self.temperature
            )
        elif self.llm_type == "ollama":
            return ChatOllama(
                model=self.model_name,
                temperature=self.temperature
            )
        else:
            raise ValueError("Unsupported LLM type. Choose 'openai' or 'ollama'.")

    def reset_state(
        self,
        llm: ChatOpenAI | ChatOllama,
    ) -> HealthBotState:
        """
        Reset the state of the HealthBot.
        """
        # Clear previous health information to maintain privacy
        return HealthBotState(llm=llm)

    def parse_quiz(
        self,
        quiz_text: str,
    ) -> tuple[str, list[str]]:
        """
        Parse the quiz text into a question and options.
        """
        # Simple parser to split question and options
        lines = quiz_text.strip().split('\n')
        question = ""
        options = []
        for line in lines:
            if line.startswith("a)") or line.startswith("b)") or line.startswith("c)") or line.startswith("d)"):
                options.append(line)
            elif line.lower().startswith("question:"):
                question = line[len("Question:"):].strip()
            elif line and not line.startswith("summary:"):
                question += " " + line.strip()
        
        return question, options


In [8]:
# /usr/bin/env python3
# healthAiBot/healthaibot/utils/agent_utils.py
"""
Utility functions for HealthBot agent operations.
"""


from typing import Optional, Callable
from langchain_tavily import TavilySearch


def tavily_search_tool(topic: str) -> str:
    """Search for medical information from trusted sources like NIH, Mayo Clinic, and WebMD."""
    search = TavilySearch()
    query = f"{topic} site:nih.gov OR site:mayoclinic.org OR site:webmd.com"
    return search.invoke(query)


class GraphHelper:
    """
    Helper class for managing graph-related operations.
    """
    def __init__(
        self,
    ) -> None:
        """
        Initialize with the current state.
        """

    def ask_patient(
        self,
        state: HealthBotState,
    ) -> HealthBotState:
        """
        Prompt the user for a health topic and update the state.
        """
        try:
            topic = input("What health topic or medical condition would you like to learn about? ")
        except EOFError:
            print("\nInput ended unexpectedly. Exiting HealthBot.")
            exit(0)
        state.topic = topic
        print(f"You have chosen to learn about: {state.topic}")
        # Add a user message for the tool node
        state.messages = [
            {"role": "user", "content": f"I want to learn about {state.topic}."}
        ]
        return state

    def generate_assistant_message(
        self,
        state: HealthBotState,
    ) -> HealthBotState:
        """
        Generate an assistant message after user input, required for ToolNode.
        """
        # Set context and messages
        state.messages = [
            {"role": "system", "content": "You are a helpful medical information assistant."},
            {"role": "user", "content": f"I want to learn about {state.topic}."},
            {"role": "assistant", "content": f"I'll search for accurate information about {state.topic} from reliable medical sources. Let me use the search tool to find relevant details."}
        ]
        return state

    def ask_for_focus(
        self,
        state: HealthBotState,
    ) -> HealthBotState:
        """
        Ask the user if they want to focus on a specific aspect.
        """
        if not state.focus:
            try:
                focus = input("Do you want to focus on a specific aspect (e.g., symptoms, treatment, prevention)? If yes, enter it, otherwise press Enter: ")
            except EOFError:
                print("\nInput ended unexpectedly. Using no specific focus.")
                focus = ""
            if focus.strip():
                state.focus = focus.strip()
        return state

    def ask_for_focus(
        self,
        state: HealthBotState,
    ) -> HealthBotState:
        """
        Ask the user if they want to focus on a specific aspect.
        """
        if not state.focus:
            try:
                focus = input("Do you want to focus on a specific aspect (e.g., symptoms, treatment, prevention)? If yes, enter it, otherwise press Enter: ")
            except EOFError:
                print("\nInput ended unexpectedly. Using no specific focus.")
                focus = ""
            if focus.strip():
                state.focus = focus.strip()
        return state

    def search_tavily(
        self,
        state: HealthBotState,
    ) -> HealthBotState:
        """
        Search for relevant information using the Tavily ToolNode.
        Record tool call event in state.
        """
        # Example event recording (actual tool call handled by ToolNode)
        event = {
            "event": "tool_call",
            "tool": "tavily_search_tool",
            "topic": state.topic
        }
        state.tool_call_events.append(event)
        return state

    def summarize_results(
        self,
        state: HealthBotState,
    ) -> HealthBotState:
        """
        Summarize the search results using the LLM.
        Enforce: summary must be exactly 3–4 paragraphs, use no outside knowledge, and be strictly based on tool output.
        """
        llm = state.llm
        focus = getattr(state, 'focus', None)
        base_prompt = (
            "Summarize the following medical information for a patient in simple, friendly language. "
            "Your summary must be exactly 3–4 paragraphs. "
            "Use no outside knowledge; only use the information provided below. "
            "Ground every statement in the provided search results. "
            "If the user requests a specific focus (e.g., symptoms, treatment, prevention), focus on that aspect. "
        )
        if focus:
            base_prompt += f"Focus on: {focus}. "
        prompt = base_prompt + f"\nSearch Results:\n{state.search_results}"
        summary = llm.invoke(prompt)
        # Extract content from AIMessage if needed
        if hasattr(summary, 'content'):
            state.summary = summary.content
        else:
            state.summary = str(summary)
        return state

    def present_summary(
        self,
        state: HealthBotState,
    ) -> HealthBotState:
        """
        Present the summarized information to the user.
        """
        print("\nHere is a summary of what you asked about:\n")
        print(state.summary)
        return state

    def comprehension_prompt(
        self,
        state: HealthBotState,
    ) -> HealthBotState:
        """
        Prompt the user for a comprehension check.
        """
        try:
            input("\nPress Enter when you are ready to take a comprehension check.")
        except EOFError:
            print("\nInput ended unexpectedly. Proceeding with comprehension check.")
        return state

    def create_quiz(
        self,
        state: HealthBotState,
    ) -> HealthBotState:
        """
        Create a quiz based on the current state.
        Enforce: quiz must be based only on the summary, single question, and output format.
        """
        llm = state.llm
        previous_questions = getattr(state, 'previous_questions', [])
        prompt = (
            "Create ONE multiple-choice quiz question that is directly relevant to the following summary. "
            "Base the question ONLY on the summary below. Do NOT use any outside knowledge. "
            "The question should have exactly 4 distinct answer options labeled a), b), c), and d). "
            "Do NOT reveal the correct answer. "
            "Do NOT repeat previous questions. "
            "Format your response as:\n"
            "Question:\n<your single question>\n"
            "a) <option 1>\n"
            "b) <option 2>\n"
            "c) <option 3>\n"
            "d) <option 4>\n"
            f"Summary: {state.summary}\n"
            f"Previous questions: {previous_questions}"
        )
        quiz_question = llm.invoke(prompt)
        # Extract content from AIMessage if needed
        if hasattr(quiz_question, 'content'):
            state.quiz_question = quiz_question.content
        else:
            state.quiz_question = str(quiz_question)
        return state

    def present_quiz(
        self,
        state: HealthBotState,
    ) -> HealthBotState:
        """
        Present the quiz question to the user.
        """
        print("\nQuiz Question:\n")
        print(state.quiz_question)
        return state

    def get_quiz_answer(
        self,
        state: HealthBotState,
    ) -> HealthBotState:
        """
        Get the user's answer to the quiz question.
        """
        try:
            answer = input("\nEnter your answer to the quiz question: ")
        except EOFError:
            print("\nInput ended unexpectedly. Exiting HealthBot.")
            exit(0)
        state.quiz_answer = answer
        return state

    def grade_quiz(
        self,
        state: HealthBotState,
    ) -> HealthBotState:
        """
        Grade the user's answer to the quiz question.
        Enforce: use only the summary, output letter grade (A–F) plus brief justification.
        """
        llm = state.llm
        prompt = (
            "Grade the following answer to the quiz question. "
            "Use ONLY the summary below as your data source. "
            "Respond with a letter grade (A, B, C, D, or F) and a brief justification (1–2 sentences) referencing the summary and the correct answer. "
            "Format your response as:\n"
            "Grade: <A–F>\nJustification: <brief explanation>\n"
            f"Summary: {state.summary}\n"
            f"Question: {state.quiz_question}\n"
            f"Answer: {state.quiz_answer}"
        )
        grading = llm.invoke(prompt)
        # Extract content from AIMessage if needed
        if hasattr(grading, 'content'):
            state.grading = grading.content
        else:
            state.grading = str(grading)
        return state

    def present_feedback(
        self,
        state: HealthBotState,
    ) -> HealthBotState:
        """
        Present the feedback to the user.
        """
        print("\nYour grade and feedback:\n")
        print(state.grading)
        # Don't prompt for next action here - let the CLI handle it
        return state

In [9]:
# /usr/bin/env python3
"""
healthAiBot graph definition.
"""


from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode


# Feedback router for conditional graph edges after present_feedback
def feedback_router(state: HealthBotState):
    if state.continue_flag == 'quiz':
        return "create_quiz"
    elif state.continue_flag == 'new':
        return "ask_patient"
    else:
        return END

def build_healthbot_graph(model) -> StateGraph:
    """
    Build the HealthBot graph with nodes and transitions, using HealthBotState and ToolNode for Tavily search.
    """
    helper = GraphHelper()
    graph = StateGraph(HealthBotState)

    # Create a custom search function that works with the state
    def search_tavily_node(state: HealthBotState) -> HealthBotState:
        """Execute Tavily search and store results in state."""
        try:
            results = tavily_search_tool(state.topic)
            state.search_results = str(results)
        except Exception as e:
            state.search_results = f"Error searching for {state.topic}: {str(e)}"
        return state

    # Add all nodes to the graph
    graph.add_node("ask_patient", helper.ask_patient)
    graph.add_node("generate_assistant_message", helper.generate_assistant_message)
    graph.add_node("search_tavily", search_tavily_node)
    graph.add_node("ask_for_focus", helper.ask_for_focus)
    graph.add_node("summarize_results", helper.summarize_results)
    graph.add_node("present_summary", helper.present_summary)
    graph.add_node("comprehension_prompt", helper.comprehension_prompt)
    graph.add_edge("ask_patient", "generate_assistant_message")
    graph.add_edge("generate_assistant_message", "search_tavily")
    graph.add_edge("search_tavily", "ask_for_focus")
    graph.add_edge("ask_for_focus", "summarize_results")
    graph.add_edge("summarize_results", "present_summary")
    graph.add_edge("present_summary", "comprehension_prompt")
    # End the graph after comprehension_prompt - let CLI handle the quiz loop
    graph.set_entry_point("ask_patient")

    return graph


In [16]:
# /usr/bin/env python3
"""
HealthBot Command Line Interface (CLI)
This script provides a command-line interface for interacting with the HealthBot application.
Users can specify various parameters such as the LLM backend, model name, and temperature.
"""

#import argparse


def main():
    """
    Main function to run the HealthBot CLI.
    """
    '''
    parser = argparse.ArgumentParser(description="HealthBot CLI")
    parser.add_argument(
        '--llm_type',
        choices=['openai', 'ollama'],
        default='ollama',
        help='Choose LLM backend: openai or ollama'
    )
    parser.add_argument(
        '--model_name',
        type=str,
        default='gemma3:1b',
        help='Model name for LLM'
    )
    parser.add_argument(
        '--temperature',
        type=float,
        default=0.3,
        help='Temperature for LLM'
    )
    # Add more arguments as needed
    args = parser.parse_args()
    '''

    llm_type = "ollama"
    model_name = "gemma3:1b"
    temperature = 0.3

    healthbot = HealthBotUtils(
        llm_type=llm_type,
        model_name=model_name,
        temperature=temperature,
    )
    
    llm = healthbot.get_llm()


    graph = build_healthbot_graph(llm)
    app = graph.compile()

    print("Welcome to HealthBot!")
    while True:
        state = healthbot.reset_state(llm)
        # Run the graph workflow to execute the full flow including focus question
        state = app.invoke(state)

        # The graph already handled focus, search, summarization, and summary presentation
        # No need to duplicate these steps here
        
        # Convert state back to HealthBotState if it's a dict
        if isinstance(state, dict):
            from healthaibot.utils.utils import HealthBotState
            state = HealthBotState(**state)

        # Track previous questions for this topic
        if not hasattr(state, 'previous_questions'):
            state.previous_questions = []
        
        # Create GraphHelper for quiz operations
        graphhelper = GraphHelper()
        quiz_active = True
        while quiz_active:
            # Create and present quiz
            state = graphhelper.create_quiz(state)
            question, options = healthbot.parse_quiz(state.quiz_question)
            print("\nQuiz Question:\n" + question)
            for opt in options:
                print(opt)
            state.previous_questions.append(question)
            state = graphhelper.get_quiz_answer(state)
            state = graphhelper.grade_quiz(state)
            state = graphhelper.present_feedback(state)

            try:
                next_action = input("Would you like to take another quiz on this topic (enter 'quiz'), learn about a new topic (enter 'new'), or exit (enter 'exit')? ")
            except EOFError:
                print("\nInput ended unexpectedly. Thank you for using HealthBot. Stay healthy!")
                return
            if next_action.lower() == 'quiz':
                print("Let's take another quiz on this topic!")
                continue  # Stay in quiz loop
            elif next_action.lower() == 'new':
                print("Let's learn about a new topic!")
                quiz_active = False  # Break quiz loop, go to new topic
            elif next_action.lower() == 'exit':
                print("Thank you for using HealthBot. Stay healthy!")
                return
            else:
                print("Invalid input. Please enter 'quiz', 'new', or 'exit'.")

In [17]:
main()

Welcome to HealthBot!


What health topic or medical condition would you like to learn about?  common cold


You have chosen to learn about: common cold


Do you want to focus on a specific aspect (e.g., symptoms, treatment, prevention)? If yes, enter it, otherwise press Enter:  symptoms



Here is a summary of what you asked about:

Okay, here’s a summary of the information for a patient, keeping it exactly 3-4 paragraphs and focusing on symptoms, presented in a friendly and easily understandable way:

The common cold is a very common illness that can cause a variety of symptoms, and it’s often a bit of a nuisance for everyone. It’s caused by viruses, and it usually lasts for about a week, but sometimes it can last longer.  The most common symptoms you might experience include a runny nose, congestion in your nose, and sneezing.  Sometimes, you might also feel a bit of a headache or a sore throat.

The symptoms of a cold can vary from person to person, but generally, they’re pretty noticeable.  Over-the-counter medicines like decongestants and pain relievers can help ease the discomfort, but they don’t actually cure the cold.  It’s important to remember that antibiotics don't work against viruses, so they won't help you get rid of a cold.

If you’re experiencing these s

...


Quiz Question:
 What is the primary cause of the illness described in the summary?
a) A bacterial infection
b) A fungal infection
c) A viral infection
d) A parasitic infection



Enter your answer to the quiz question:  a



Your grade and feedback:

Grade: B
Justification: The answer correctly identifies the primary cause of the illness as a viral infection. The summary explicitly states that the cold is caused by viruses and that it’s a common illness that’s often a nuisance due to the presence of symptoms like a runny nose, congestion, and sneezing. While it mentions antibiotics don’t work against viruses, the core explanation – a virus – is accurately presented.


Would you like to take another quiz on this topic (enter 'quiz'), learn about a new topic (enter 'new'), or exit (enter 'exit')?  exit


Thank you for using HealthBot. Stay healthy!
